# DWI Artifact Segmentation Training on Google Colab

This notebook trains an Attention U-Net model for artifact segmentation (red color: FF0000)

## Setup Steps:
1. Upload your data to Google Drive
2. Mount Google Drive
3. Configure paths and parameters
4. Run training
5. View results in MLflow

## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch torchvision
!pip install -q opencv-python albumentations
!pip install -q matplotlib tqdm
!pip install -q mlflow

## 2. Mount Google Drive (if data is on Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Configuration

### 📌 กำหนดค่าที่นี่!

In [ ]:
import torch
from pathlib import Path

# ==================== ⚙️ CONFIGURE HERE ====================

# 📂 Base Path (ปรับให้ตรงกับ working directory ของคุณ)
BASE_PATH = Path("/content/drive/MyDrive/AiiLAB/_ProjectCommercial/_DWI-DBMSPattaya")

# 📂 Data Paths - ใช้ relative path จาก BASE_PATH
MASKS_DIR = BASE_PATH / "T4-ARTFSEG/Dataset-ArfifactLabel"  # Path to mask folder
ORIGINAL_DIR = BASE_PATH / "T2-ARTIFACT_CLASSIFY/dataset-t2-24jul/Artifact"  # Path to original images folder
MODELS_DIR = BASE_PATH / "T4-ARTFSEG/models"  # Where to save models

# 🎨 Artifact Color (Red: FF0000)
ARTIFACT_COLOR = [255, 0, 0]  # RGB

# 📐 Image Size
IMAGE_SIZE = (384, 384)

# 📊 Data Split Ratio
TRAIN_RATIO = 0.80
VAL_RATIO = 0.15
TEST_RATIO = 0.05
RANDOM_SEED = 42

# 🏗️ Model Architecture
IN_CHANNELS = 3      # RGB input
OUT_CHANNELS = 1     # Binary segmentation
BASE_CHANNELS = 64   # Model size (64=~31M params, 32=~8M params)

# 🎯 Training Parameters
EPOCHS = 100
BATCH_SIZE = 16  # ลดถ้า Out of Memory
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
EARLY_STOP_PATIENCE = 20

# 🔄 Augmentation
USE_AUGMENTATION = False  # True = enable augmentation
AUG_HFLIP_PROB = 0.3
AUG_ROTATE_LIMIT = 10
AUG_ROTATE_PROB = 0.25
AUG_BRIGHTNESS_LIMIT = 0.1
AUG_BRIGHTNESS_PROB = 0.2

# 📝 MLflow
MLFLOW_EXPERIMENT_NAME = "DWI-Artifact-Colab"

# 💻 Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = True if torch.cuda.is_available() else False

# ==================== END CONFIGURATION ====================

# Create directories
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Print configuration
print("="*60)
print("📋 CONFIGURATION")
print("="*60)
print(f"Base Path: {BASE_PATH}")
print(f"Masks Dir: {MASKS_DIR}")
print(f"  → Exists: {MASKS_DIR.exists()}")
print(f"Original Dir: {ORIGINAL_DIR}")
print(f"  → Exists: {ORIGINAL_DIR.exists()}")
print(f"Models Dir: {MODELS_DIR}")
print(f"\nImage Size: {IMAGE_SIZE}")
print(f"Artifact Color: RGB{ARTIFACT_COLOR}")
print(f"\nSplit: {TRAIN_RATIO:.0%} train / {VAL_RATIO:.0%} val / {TEST_RATIO:.0%} test")
print(f"\nModel: Attention U-Net")
print(f"Base Channels: {BASE_CHANNELS}")
print(f"\nTraining:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Learning Rate: {LEARNING_RATE}")
print(f"  Device: {DEVICE}")
print(f"  Mixed Precision: {USE_AMP}")
print(f"\nAugmentation: {'ENABLED ✓' if USE_AUGMENTATION else 'DISABLED ✗'}")
print("="*60)

# แสดงไฟล์ตัวอย่างใน MASKS_DIR
if MASKS_DIR.exists():
    import glob
    sample_files = glob.glob(str(MASKS_DIR / "*.png"))[:5]
    if sample_files:
        print(f"\n✓ Found PNG files in MASKS_DIR:")
        for f in sample_files:
            print(f"  - {Path(f).name}")
    else:
        print(f"\n⚠️ No PNG files found in {MASKS_DIR}")
        print("Please check the path!")
else:
    print(f"\n❌ MASKS_DIR does not exist: {MASKS_DIR}")
    print("Please check the path!")

## 4. Import Model Definition

Copy from model.py

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    """Standard Conv Block: Conv -> BN -> ReLU -> Conv -> BN -> ReLU"""
    
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        return x


class AttentionGate(nn.Module):
    """Attention Gate for focusing on relevant features"""
    
    def __init__(self, F_g, F_l, F_int):
        super(AttentionGate, self).__init__()
        
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi


class AttentionUNet(nn.Module):
    """Attention U-Net for artifact segmentation"""
    
    def __init__(self, in_channels=3, out_channels=1, base_channels=64):
        super(AttentionUNet, self).__init__()
        
        # Encoder
        self.enc1 = ConvBlock(in_channels, base_channels)
        self.pool1 = nn.MaxPool2d(2)
        
        self.enc2 = ConvBlock(base_channels, base_channels * 2)
        self.pool2 = nn.MaxPool2d(2)
        
        self.enc3 = ConvBlock(base_channels * 2, base_channels * 4)
        self.pool3 = nn.MaxPool2d(2)
        
        self.enc4 = ConvBlock(base_channels * 4, base_channels * 8)
        self.pool4 = nn.MaxPool2d(2)
        
        # Bottleneck
        self.bottleneck = ConvBlock(base_channels * 8, base_channels * 16)
        
        # Decoder with Attention Gates
        self.up4 = nn.ConvTranspose2d(base_channels * 16, base_channels * 8, kernel_size=2, stride=2)
        self.att4 = AttentionGate(F_g=base_channels * 8, F_l=base_channels * 8, F_int=base_channels * 4)
        self.dec4 = ConvBlock(base_channels * 16, base_channels * 8)
        
        self.up3 = nn.ConvTranspose2d(base_channels * 8, base_channels * 4, kernel_size=2, stride=2)
        self.att3 = AttentionGate(F_g=base_channels * 4, F_l=base_channels * 4, F_int=base_channels * 2)
        self.dec3 = ConvBlock(base_channels * 8, base_channels * 4)
        
        self.up2 = nn.ConvTranspose2d(base_channels * 4, base_channels * 2, kernel_size=2, stride=2)
        self.att2 = AttentionGate(F_g=base_channels * 2, F_l=base_channels * 2, F_int=base_channels)
        self.dec2 = ConvBlock(base_channels * 4, base_channels * 2)
        
        self.up1 = nn.ConvTranspose2d(base_channels * 2, base_channels, kernel_size=2, stride=2)
        self.att1 = AttentionGate(F_g=base_channels, F_l=base_channels, F_int=base_channels // 2)
        self.dec1 = ConvBlock(base_channels * 2, base_channels)
        
        # Output
        self.out = nn.Conv2d(base_channels, out_channels, kernel_size=1)
    
    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        e3 = self.enc3(self.pool2(e2))
        e4 = self.enc4(self.pool3(e3))
        
        # Bottleneck
        b = self.bottleneck(self.pool4(e4))
        
        # Decoder with Attention
        d4 = self.up4(b)
        e4_att = self.att4(g=d4, x=e4)
        d4 = torch.cat([e4_att, d4], dim=1)
        d4 = self.dec4(d4)
        
        d3 = self.up3(d4)
        e3_att = self.att3(g=d3, x=e3)
        d3 = torch.cat([e3_att, d3], dim=1)
        d3 = self.dec3(d3)
        
        d2 = self.up2(d3)
        e2_att = self.att2(g=d2, x=e2)
        d2 = torch.cat([e2_att, d2], dim=1)
        d2 = self.dec2(d2)
        
        d1 = self.up1(d2)
        e1_att = self.att1(g=d1, x=e1)
        d1 = torch.cat([e1_att, d1], dim=1)
        d1 = self.dec1(d1)
        
        # Output
        out = self.out(d1)
        return torch.sigmoid(out)


def count_parameters(model):
    """Count total and trainable parameters"""
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable


print("✅ Model classes loaded successfully!")

## 5. Import Training Functions

Copy from train.py

In [ ]:
import os
import glob
import json
import tempfile
from datetime import datetime

import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

import mlflow
import mlflow.pytorch


# ==================== Dataset ====================

class ArtifactDataset(Dataset):
    """Simple Artifact Dataset with RGB images"""
    
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.transform = transform
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        image = self.images[idx]  # (H, W, 3) - RGB
        mask = self.masks[idx]    # (H, W)
        
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
            mask = mask.unsqueeze(0).float()
        else:
            image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0
            mask = torch.from_numpy(mask).unsqueeze(0).float()
        
        return image, mask


# ==================== Loss Function ====================

class DiceLoss(nn.Module):
    """Simple Dice Loss"""
    
    def __init__(self, smooth=1e-6):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, pred, target):
        pred = pred.view(-1)
        target = target.view(-1)
        
        intersection = (pred * target).sum()
        dice = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        
        return 1 - dice


def calculate_dice_score(pred, target):
    """Calculate Dice score (metric)"""
    pred = pred.view(-1)
    target = target.view(-1)
    
    intersection = (pred * target).sum()
    dice = (2. * intersection + 1e-6) / (pred.sum() + target.sum() + 1e-6)
    
    return dice.item()


def calculate_iou(pred, target):
    """Calculate IoU (Jaccard Index)"""
    pred = pred.view(-1)
    target = target.view(-1)
    
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    iou = (intersection + 1e-6) / (union + 1e-6)
    
    return iou.item()


print("✅ Training functions loaded successfully!")

## 6. Data Loading Functions

In [ ]:
def extract_red_mask(mask_rgb, artifact_color=[255, 0, 0], tolerance=10):
    """Extract binary mask from red artifacts in RGB mask"""
    lower = np.array([max(0, c - tolerance) for c in artifact_color])
    upper = np.array([min(255, c + tolerance) for c in artifact_color])
    
    binary_mask = cv2.inRange(mask_rgb, lower, upper)
    binary_mask = (binary_mask > 0).astype(np.float32)
    
    return binary_mask


def load_and_preprocess_data():
    """Load PNG files and prepare datasets"""
    print("\n" + "="*60)
    print("📂 Loading and Preprocessing Data")
    print("="*60)
    
    # Find all mask files
    mask_files = sorted(glob.glob(str(MASKS_DIR / "*.png")))
    print(f"Found {len(mask_files)} mask files in {MASKS_DIR}")
    
    if len(mask_files) == 0:
        raise FileNotFoundError(f"No .png files found in {MASKS_DIR}")
    
    # Load all images and extract masks
    all_images = []
    all_masks = []
    skipped = 0
    
    print("\nLoading PNG files...")
    for mask_path in tqdm(mask_files, desc="Loading"):
        mask_name = Path(mask_path).name
        original_path = ORIGINAL_DIR / mask_name
        
        if not original_path.exists():
            print(f"Warning: Original not found for {mask_name}, skipping...")
            skipped += 1
            continue
        
        # Load images
        original_img = cv2.imread(str(original_path))
        mask_img = cv2.imread(str(mask_path))
        
        if original_img is None or mask_img is None:
            print(f"Warning: Failed to load {mask_name}, skipping...")
            skipped += 1
            continue
        
        # Convert BGR to RGB
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
        mask_img = cv2.cvtColor(mask_img, cv2.COLOR_BGR2RGB)
        
        # Extract red artifact mask
        binary_mask = extract_red_mask(mask_img, ARTIFACT_COLOR)
        
        # Skip if no artifact found
        if binary_mask.sum() == 0:
            skipped += 1
            continue
        
        # Resize
        original_img = cv2.resize(original_img, IMAGE_SIZE)
        binary_mask = cv2.resize(binary_mask, IMAGE_SIZE, interpolation=cv2.INTER_NEAREST)
        binary_mask = (binary_mask > 0.5).astype(np.float32)
        
        all_images.append(original_img)
        all_masks.append(binary_mask)
    
    # Convert to numpy arrays
    all_images = np.array(all_images, dtype=np.uint8)
    all_masks = np.array(all_masks, dtype=np.float32)
    
    print(f"\nTotal images loaded: {len(all_images)}")
    print(f"Skipped: {skipped}")
    print(f"Image shape: {all_images.shape}")
    print(f"Mask shape: {all_masks.shape}")
    
    if len(all_images) == 0:
        raise ValueError("No valid images loaded. Check your data!")
    
    # Split into train/val/test
    np.random.seed(RANDOM_SEED)
    indices = np.random.permutation(len(all_images))
    
    n_train = int(len(indices) * TRAIN_RATIO)
    n_val = int(len(indices) * VAL_RATIO)
    
    train_idx = indices[:n_train]
    val_idx = indices[n_train:n_train + n_val]
    test_idx = indices[n_train + n_val:]
    
    train_images = all_images[train_idx]
    train_masks = all_masks[train_idx]
    val_images = all_images[val_idx]
    val_masks = all_masks[val_idx]
    test_images = all_images[test_idx]
    test_masks = all_masks[test_idx]
    
    print(f"\nSplit:")
    print(f"  Train: {len(train_images)} images")
    print(f"  Val: {len(val_images)} images")
    print(f"  Test: {len(test_images)} images")
    
    # Create augmentation transforms
    if USE_AUGMENTATION:
        print("\n✓ Augmentation ENABLED for training")
        train_transform = A.Compose([
            A.HorizontalFlip(p=AUG_HFLIP_PROB),
            A.Rotate(limit=AUG_ROTATE_LIMIT, p=AUG_ROTATE_PROB),
            A.RandomBrightnessContrast(
                brightness_limit=AUG_BRIGHTNESS_LIMIT,
                contrast_limit=AUG_BRIGHTNESS_LIMIT,
                p=AUG_BRIGHTNESS_PROB
            ),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    else:
        print("\n✗ Augmentation DISABLED")
        train_transform = A.Compose([
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ToTensorV2()
        ])
    
    val_transform = A.Compose([
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
    # Create datasets
    train_dataset = ArtifactDataset(train_images, train_masks, transform=train_transform)
    val_dataset = ArtifactDataset(val_images, val_masks, transform=val_transform)
    test_dataset = ArtifactDataset(test_images, test_masks, transform=val_transform)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2, pin_memory=True)
    
    print("="*60 + "\n")
    
    return train_loader, val_loader, test_loader


print("✅ Data loading functions ready!")

## 7. Training Functions

In [ ]:
def train_one_epoch(model, dataloader, criterion, optimizer, scaler, device):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    running_dice = 0.0
    
    pbar = tqdm(dataloader, desc="Training")
    for images, masks in pbar:
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad()
        
        if USE_AMP:
            with torch.amp.autocast('cuda'):
                outputs = model(images)
                loss = criterion(outputs, masks)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
        
        with torch.no_grad():
            preds = (outputs > 0.5).float()
            dice = calculate_dice_score(preds, masks)
        
        running_loss += loss.item()
        running_dice += dice
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}', 'dice': f'{dice:.4f}'})
    
    avg_loss = running_loss / len(dataloader)
    avg_dice = running_dice / len(dataloader)
    
    return avg_loss, avg_dice


def validate(model, dataloader, criterion, device):
    """Validate the model"""
    model.eval()
    running_loss = 0.0
    running_dice = 0.0
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc="Validation")
        for images, masks in pbar:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            preds = (outputs > 0.5).float()
            dice = calculate_dice_score(preds, masks)
            
            running_loss += loss.item()
            running_dice += dice
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'dice': f'{dice:.4f}'})
    
    avg_loss = running_loss / len(dataloader)
    avg_dice = running_dice / len(dataloader)
    
    return avg_loss, avg_dice


print("✅ Training functions ready!")

## 8. Load Data

In [ ]:
# Set random seeds
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Load data
train_loader, val_loader, test_loader = load_and_preprocess_data()

## 9. Create Model

In [ ]:
print("="*60)
print("🏗️  Creating Model")
print("="*60)

model = AttentionUNet(
    in_channels=IN_CHANNELS,
    out_channels=OUT_CHANNELS,
    base_channels=BASE_CHANNELS
).to(DEVICE)

total_params, trainable_params = count_parameters(model)
print(f"Total parameters: {total_params:,} ({total_params/1e6:.1f}M)")
print(f"Trainable parameters: {trainable_params:,} ({trainable_params/1e6:.1f}M)")
print("="*60 + "\n")

## 10. Setup Training

In [ ]:
# Loss, optimizer, scheduler
criterion = DiceLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5)

# Mixed precision scaler
scaler = torch.amp.GradScaler('cuda') if USE_AMP else None

# Setup MLflow
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

print("✅ Training setup complete!")

## 11. Start Training 🚀

In [ ]:
run_name = f"artifact_colab_{datetime.now():%Y%m%d_%H%M%S}"

with mlflow.start_run(run_name=run_name):
    print("="*60)
    print("🔬 MLflow Tracking Initialized")
    print("="*60)
    print(f"Experiment: {MLFLOW_EXPERIMENT_NAME}")
    print(f"Run Name: {run_name}")
    print("="*60 + "\n")
    
    # Log parameters
    mlflow.log_param("image_size", IMAGE_SIZE)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("base_channels", BASE_CHANNELS)
    mlflow.log_param("train_samples", len(train_loader.dataset))
    mlflow.log_param("val_samples", len(val_loader.dataset))
    mlflow.log_param("test_samples", len(test_loader.dataset))
    mlflow.log_param("model_params", total_params)
    mlflow.log_param("use_augmentation", USE_AUGMENTATION)
    mlflow.log_param("artifact_color", ARTIFACT_COLOR)
    mlflow.log_param("platform", "Google Colab")
    
    # Training loop
    print("="*60)
    print("🚀 Starting Training")
    print("="*60 + "\n")
    
    best_dice = 0.0
    patience_counter = 0
    history = {'train_loss': [], 'train_dice': [], 'val_loss': [], 'val_dice': []}
    
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")
        print("-" * 60)
        
        # Train
        train_loss, train_dice = train_one_epoch(model, train_loader, criterion, optimizer, scaler, DEVICE)
        
        # Validate
        val_loss, val_dice = validate(model, val_loader, criterion, DEVICE)
        
        # Update scheduler
        scheduler.step(val_dice)
        current_lr = optimizer.param_groups[0]['lr']
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_dice'].append(train_dice)
        history['val_loss'].append(val_loss)
        history['val_dice'].append(val_dice)
        
        # Log to MLflow
        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("train_dice", train_dice, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("val_dice", val_dice, step=epoch)
        mlflow.log_metric("learning_rate", current_lr, step=epoch)
        
        # Print summary
        print(f"\nResults:")
        print(f"  Train Loss: {train_loss:.4f} | Train Dice: {train_dice:.4f}")
        print(f"  Val Loss: {val_loss:.4f} | Val Dice: {val_dice:.4f}")
        print(f"  Learning Rate: {current_lr:.6f}")
        
        # Save best model
        if val_dice > best_dice:
            best_dice = val_dice
            patience_counter = 0
            torch.save(model.state_dict(), MODELS_DIR / "best_model.pth")
            print(f"  ✅ New best model saved! (Dice: {best_dice:.4f})")
        else:
            patience_counter += 1
            print(f"  No improvement ({patience_counter}/{EARLY_STOP_PATIENCE})")
        
        # Early stopping
        if patience_counter >= EARLY_STOP_PATIENCE:
            print(f"\n⏹️  Early stopping triggered after {epoch+1} epochs")
            break
    
    print("\n" + "="*60)
    print("✅ Training Completed!")
    print("="*60)
    print(f"Best Val Dice: {best_dice:.4f}")
    print("="*60 + "\n")
    
    # Log best model
    mlflow.log_artifact(str(MODELS_DIR / "best_model.pth"))
    mlflow.log_metric("best_val_dice", best_dice)

print("\n🎉 Training finished!")

## 12. Plot Training Curves

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

epochs = range(1, len(history['train_loss']) + 1)

# Loss
axes[0].plot(epochs, history['train_loss'], 'b-', label='Train Loss')
axes[0].plot(epochs, history['val_loss'], 'r-', label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Dice
axes[1].plot(epochs, history['train_dice'], 'b-', label='Train Dice')
axes[1].plot(epochs, history['val_dice'], 'r-', label='Val Dice')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Dice Score')
axes[1].set_title('Training and Validation Dice Score')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(MODELS_DIR / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## 13. Evaluate on Test Set

In [ ]:
# Load best model
model.load_state_dict(torch.load(MODELS_DIR / "best_model.pth"))
model.eval()

print("\n" + "="*60)
print("📊 Evaluating on Test Set")
print("="*60)

all_dice = []
all_iou = []

with torch.no_grad():
    for images, masks in tqdm(test_loader, desc="Testing"):
        images = images.to(DEVICE)
        masks = masks.to(DEVICE)
        
        outputs = model(images)
        preds = (outputs > 0.5).float()
        
        dice = calculate_dice_score(preds, masks)
        iou = calculate_iou(preds, masks)
        
        all_dice.append(dice)
        all_iou.append(iou)

mean_dice = np.mean(all_dice)
std_dice = np.std(all_dice)
mean_iou = np.mean(all_iou)
std_iou = np.std(all_iou)

print(f"\nTest Results:")
print(f"  Dice Score: {mean_dice:.4f} ± {std_dice:.4f}")
print(f"  IoU Score: {mean_iou:.4f} ± {std_iou:.4f}")
print("="*60 + "\n")

## 14. Visualize Predictions

In [ ]:
# Get a few test samples
model.eval()
test_iter = iter(test_loader)

fig, axes = plt.subplots(3, 3, figsize=(15, 15))

for idx in range(min(3, len(test_loader))):
    images, masks = next(test_iter)
    images = images.to(DEVICE)
    masks = masks.to(DEVICE)
    
    with torch.no_grad():
        outputs = model(images)
        preds = (outputs > 0.5).float()
    
    # Denormalize image
    img_np = images[0].cpu().numpy().transpose(1, 2, 0)
    img_np = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img_np = np.clip(img_np, 0, 1)
    
    mask_np = masks[0, 0].cpu().numpy()
    pred_np = preds[0, 0].cpu().numpy()
    
    dice = calculate_dice_score(preds, masks)
    
    # Image
    axes[idx, 0].imshow(img_np)
    axes[idx, 0].set_title('Input Image')
    axes[idx, 0].axis('off')
    
    # Ground Truth
    axes[idx, 1].imshow(img_np)
    axes[idx, 1].imshow(mask_np, cmap='Reds', alpha=0.5)
    axes[idx, 1].set_title('Ground Truth')
    axes[idx, 1].axis('off')
    
    # Prediction
    axes[idx, 2].imshow(img_np)
    axes[idx, 2].imshow(pred_np, cmap='Greens', alpha=0.5)
    axes[idx, 2].set_title(f'Prediction (Dice: {dice:.4f})')
    axes[idx, 2].axis('off')

plt.tight_layout()
plt.savefig(MODELS_DIR / 'test_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

## 15. Download Model (Optional)

In [ ]:
# Download best model to local machine
from google.colab import files

print("Downloading best model...")
files.download(str(MODELS_DIR / 'best_model.pth'))

print("Downloading training curves...")
files.download(str(MODELS_DIR / 'training_curves.png'))

print("Downloading predictions...")
files.download(str(MODELS_DIR / 'test_predictions.png'))

## 🎉 Done!

Your model has been trained successfully!

### Results:
- Best model saved to: `{MODELS_DIR}/best_model.pth`
- Training curves: `{MODELS_DIR}/training_curves.png`
- Test predictions: `{MODELS_DIR}/test_predictions.png`

### Next Steps:
1. Download the model files
2. Use the model for inference on new images
3. Fine-tune with different hyperparameters if needed